# Week 1 Skills Assessment - Case Study


Add your notes here

# Imports

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd # ensure v = 0.24 !pip install --upgrade pandas 
import numpy as np
import datetime as dt

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap,LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches

In [5]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [6]:
myColors = [
     (88/255.,166/255.,24/255.)
          ,(233/255.,131/255.,0/255.)
          ,(0/255.,117/255.,191/255.)
          ,(147/255.,38/255.,143/255.)
         ]

name = 'dexmap'
cm = LinearSegmentedColormap.from_list(name, myColors, N=4)

In [7]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [8]:
sns.set_style('darkgrid')

In [9]:
sns.set_context('talk')

# Data Import & Preprocessing

Complete the following:
- Import 'user_accounts_data.csv' as a pandas dataframe
- Import 'user_exercise_logs_data.csv' as a pandas dataframe
- Determine if any columns have nulls in either dataset
- Create the following new columns:
    - 'start_date': a date-column representing the date the exercise log was initiated
    - 'end_date': a date-column representing the date the exercise log was completed

# Basic Computations

Answer each of the following questions, with your answer

1. How many unique users are there?
2. What proportion of users are mae vs female?
3. What is the most common type of primary-device?
4. What is the average number of users per account?
5. 

# 3. EDA - Variant vs Features

Examining streaming data for insights to inform quality of statistical testing

#### Findings
  - Mean event duration (mins): 22.23 mins
  - 99th percentile (upper limit) of session duration: 145.19 mins     
       - Mean duration with outliers removed: 20.02 mins
  - 42% of events are < 5 minutes

#### Frequency of event-duration-bins

In [ ]:
fig,ax = plt.subplots(figsize=(12,7))
sns.countplot(streams_df['stream_time_bin'],hue=streams_df['variant_id'])
ax.set_xlabel('')
ax.set_yticklabels([str(int(x/1000))+'k' if x != 0 else '0' for x in np.arange(0,180000,20000)])
ax.set_ylabel('Stream Event Count')
ax.set_title('Frequency of Stream Event Duration Bin')
plt.show()

In [ ]:
# Simple definition of outliers: streams with duration > 99th percentile
q99_stream_mins = np.round(streams_df['stream_time_min'].quantile(q=0.99),2)
print('Stream duration (mins) 99th Percentile: {0}'.format(q99_stream_mins))

In [ ]:
streams_df['mos_score'][streams_df['stream_time_min']>q99_stream_mins].mean()

#### MOS vs Variant & Stream Duration

In [ ]:
fig,ax = plt.subplots(figsize=(12,7))
sns.violinplot(x="variant_id", y="mos_score", hue="stream_time_bin",data=streams_df)
ax.set_ylim((0,7))
ax.set_xlabel('Variant Id')
ax.set_ylabel('MOS Score')
ax.set_title('MOS vs Vaariant & Session Duration')
ax.legend(ncol=5)
plt.show()

### Cutting stream_df with exclusions

In [ ]:
# Remove streams where: start time > end time & stream-time (mins) > 24 hours
streams_df_cut = streams_df[(streams_df['stream_time_min']<=q99_stream_mins)&\
                            (streams_df['start_timestamp']<=streams_df['end_timestamp'])&\
                            (streams_df['mos_score'].notnull())]

In [ ]:
streams_df_cut.describe()

In [ ]:
g = sns.FacetGrid(streams_df_cut, hue="variant_id",aspect=2,height=5)
g = g.map(sns.distplot, "stream_time_min",bins=22)
plt.legend(fontsize=20,title='Streaming Variant')
xticks = np.arange(0,150,5)
xticks
plt.xticks(np.arange(0,150,15))
plt.xlim((0,150))
plt.xlabel("")
plt.ylabel("Proportion")

stream_min_mean = streams_df_cut['stream_time_min'].mean()
plt.axvline(x=stream_min_mean,color='black',linestyle='--')
plt.title('Distribution of Stream Event Duration')
plt.show()

In [ ]:
mins5less = np.round(streams_df_cut[streams_df_cut['stream_time_min']<5].shape[0]/streams_df_cut.shape[0]*100,2)
print('Percent of streams < 5 mins in duration: {0}'.format(mins5less))

#### MOS 

To do: 
  - Categorical scatter between MOS & stream time
  - Categorical scatter plot between MOS & variant_id

In [ ]:
# Remove streams where: start time > end time & stream-time (mins) > 24 hours
streams_df_cut = streams_df[(streams_df['stream_time_min']<=q99_stream_mins)&\
                            (streams_df['start_timestamp']<=streams_df['end_timestamp'])&\
                            (streams_df['mos_score'].notnull())]

In [ ]:
print('Max MOS: {0}, Min MOS {1}'.format(np.round(streams_df_cut['mos_score'].max(),3),streams_df_cut['mos_score'].min()))

# 4.Contextual Relationships

Findings
  - No clear/visual relationship between Variant & MOS Score
  - No clear/visual relationship between Stream event duration & MOS Score
  - No clear/visual relationship between MOS Score & Churn
  - No clear/visual relationship between Stream event duration & Churn

In [ ]:
fig,ax = plt.subplots(figsize=(8,6))
sns.violinplot(x="variant_id", y="mos_score",data=streams_df_cut)
#ax.set_ylim((0,5))
ax.set_xlabel('Variant Id')
ax.set_ylabel('MOS Score')
ax.set_title('MOS vs Variant')
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(8,6))
sns.scatterplot(x="stream_time_min", y="mos_score",data=streams_df_cut)
#ax.set_ylim((0,5))
ax.set_xlabel('Session Duration')
ax.set_ylabel('MOS Score')
ax.set_title('MOS vs Session Duration')
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(8,6))
sns.violinplot(x="variant_id", y="mos_score", hue="stream_time_bin",hue_order = ['< 1hr','1-3 hrs'],data=streams_df_cut)
#ax.set_ylim((0,5))
ax.set_xlabel('Variant Id')
ax.set_ylabel('MOS Score')
ax.set_title('MOS vs Variant & Session Duration')
ax.legend(loc = 'center')
plt.show()

In [ ]:
streams_df_cut = streams_df_cut.merge(act_df[['account_number','churned']],on=['account_number'],how='left')

In [ ]:
streams_df_cut_grouped = streams_df_cut.groupby(['churned','account_number'],as_index=False).mean()

In [ ]:
sns.catplot(x="churned", y="mos_score", data=streams_df_cut_grouped)
#ax.set_ylim((0,5))
plt.xlabel('Churned')
plt.ylabel('MOS Score')
plt.title('Churn vs Mean MOS')
#ax.legend(loc = 'center')
plt.show()

In [ ]:
sns.catplot(x="churned", y="stream_time_min", data=streams_df_cut_grouped)
#ax.set_ylim((0,5))
plt.xlabel('Churned')
plt.ylabel('Mean Stream Event Dur.')
plt.title('Churn vs Mean Stream Event Duration')
#ax.legend(loc = 'center')
plt.show()

# 5. Experimental Analysis

Study window:
  - ~4 months (4/30-8/24) 
  
  
Account data
  - Remove accounts with streaming data records indicating both variants
      - N-old = 
      - n-new = 
      
      
Streams data cleaning:
  - Remove event without start times
  - Remove outlier events 
      - Events > 146 minutes (99 percentile is 145.2 mins)
      - Events where MOS is null, and duration < 1 minute - maybe not a valid stream/not really enough time to tell the difference

In [ ]:
# Statistical Packages imports
import scipy.stats as stats
import statsmodels.stats.api as sms
from math import ceil

from statsmodels.stats.power import TTestIndPower

from statsmodels.stats.proportion import proportions_ztest, proportion_confint

from scipy.stats import shapiro

## Final Data Prep

In [ ]:
dble_variants_streams = pd.DataFrame(streams_df.groupby(['account_number'])['variant_id'].nunique()).reset_index()
dble_variants_streams = dble_variants_streams[dble_variants_streams['variant_id']>1]

In [ ]:
# Remove streams where: 
q99_stream_mins = streams_df['stream_time_min'].quantile(0.99)
no_mos_min_thresh = 1
dvs = dble_variants_streams['account_number'].unique()
ic1 = (streams_df['stream_time_min']<=q99_stream_mins)
ic2 = (streams_df['start_timestamp']<=streams_df['end_timestamp'])
ic3 = ~((streams_df['mos_score'].isnull()) & (streams_df['stream_time_min']<=no_mos_min_thresh))
ic4 = ~(streams_df['account_number'].isin(dvs))
ic5 = ~(act_df['account_number'].isin(dvs))

streams_df_cut = streams_df[ic1 & ic2 & ic3 & ic4]
act_df_cut = act_df[ic5]

In [ ]:
final_df = streams_df_cut.merge(act_df_cut[['account_number','churned']],on=['account_number'])

In [ ]:
final_df['permutation_variant'] = np.random.randint(0,2,size=len(final_df))

## Statistical Tests 

#### Tests to run:
    - t-test
        - Normalized data
        - Down Sampled
    - Permutation test
    - Bootstrap test

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
sns.barplot(x=final_df['variant_id'],y=final_df['mos_score'],ci=68)
ax.set_ylim((0,5))
ax.set_xlabel('Variant Id')
ax.set_ylabel('Mean MOS Score')
ax.set_title('Average MOS')
plt.show()

In [ ]:
g = sns.FacetGrid(final_df, hue="variant_id",aspect=2,height=5)
g = g.map(sns.distplot, "mos_score")
plt.legend(fontsize=20,title='Streaming Variant')

#plt.xticks(np.arange(0,150,15))
#plt.xlim((0,150))
plt.xlabel("")
plt.ylabel("Proportion")

stream_mos_mean = final_df['mos_score'].mean()
plt.axvline(x=stream_mos_mean,color='black',linestyle='--')
plt.title('Distribution of MOS Values')
plt.show()

### A) Relationship b/w Variant & MOS (streaming quality)

In [ ]:
# Measure sample sizes reqiored for small,  medium & large effect sizes to be found
es_all = [0.2,0.5,0.8]
es_reqs = pd.DataFrame()
for es in es_all:
    r = hp.calc_sample_size(es)
    if es_reqs.empty:
        es_reqs = r
    else:
        es_reqs = pd.concat([es_reqs,r],ignore_index=True)
es_reqs

In [ ]:
bs_all_results = pd.DataFrame()
for ss in [25,50,75,100,200,300,394]:
    bs_temp = hp.means_test_experiment(final_df,'mos_score',sample_n=ss,bootstrap=100,comparison=True)
    if bs_all_results.empty:
        bs_all_results = bs_temp.copy(deep=True)
    else:
        bs_all_results = pd.concat([bs_all_results,bs_temp],ignore_index=True)

In [ ]:
pct_single_run_sig = np.round(bs_all_results[bs_all_results['test_description']=='original']['pct_sig'].mean(),2)
pct_bs_sig = np.round(bs_all_results[bs_all_results['test_description']=='bootstrap_mean']['pct_sig'].mean(),2)
print('Percent of single tests with significant results (Varying SS): {0}'.format(pct_single_run_sig))
print('Percent of BS tests with significant results (Varying SS): {0}'.format(pct_bs_sig))

In [ ]:
pct_single_run_sig = np.round(bs_all_results[bs_all_results['test_description']=='original']['effect_size(mean_difference)'].mean(),2)
pct_bs_sig = np.round(bs_all_results[bs_all_results['test_description']=='bootstrap_mean']['effect_size(mean_difference)'].mean(),2)
print('Average % mean group difference (Varying SS): {0}'.format(pct_single_run_sig))
print('Average % mean group difference  (Varying SS): {0}'.format(pct_bs_sig))

In [ ]:
original_mean_pct_impvt = np.round(bs_all_results[bs_all_results['test_description']=='original']['effect_size(pct_improvement)'].mean(),2)
bs_mean_pct_impvt = np.round(bs_all_results[bs_all_results['test_description']=='bootstrap_mean']['effect_size(pct_improvement)'].mean(),2)
print('Average % improvement (Varying SS): {0}'.format(original_mean_pct_impvt))
print('Average % improvement (Varying SS): {0}'.format(bs_mean_pct_impvt))

In [ ]:
original_mean_cd = np.round(bs_all_results[bs_all_results['test_description']=='original']['effect_size(cd)'].mean(),2)
bs_mean_cd = np.round(bs_all_results[bs_all_results['test_description']=='bootstrap_mean']['effect_size(cd)'].mean(),2)
print('Average Effect Size (cohen D) (Varying SS): {0}'.format(original_mean_cd))
print('Average Effect Size (cohen D) (Varying SS): {0}'.format(bs_mean_cd))

In [ ]:
bs_all_results[bs_all_results['test_description']=='original']['effect_size(pct_improvement)'].mean()

In [ ]:
bs_all_results

### B) Statistical Impact on Churn

Context / Background

    - Charter video customers Q4 2017 = 16,544 (net Y loss ~ -1.7%)
    - Charter video customer Q1 2018 = 16,422 (net Y loss ~ -1.9%)
    - Quarterly churn is typically ~ 5-7% (telecom average)
    
Statistical Tests Run    

    1) Account data - Proportions test
    2) Permutation test (check against randomness)
    3) Bootstrap test (check by resampling)

In [ ]:
act_df_cut.groupby(['variant_id'])['churned'].value_counts().plot(kind='pie',figsize=(6,6))
plt.show()

#### 1. Proportions Test

In [ ]:
#sample_size = act_df_cut.groupby('variant_id')['account_number'].nunique().min()
sample_size = 200
results_df = hp.prop_experiment(act_df_cut,sample_n=sample_size,dep_col='churned')
results_df

#### 2. Permutations Test

In [ ]:
# Permutation test
#sample_size = act_df_cut.groupby('variant_id')['account_number'].nunique().min()
sample_size = 100
nperms = 100
results_pdf = pd.DataFrame()
for i in range(1,100):
    temp_df = hp.prop_experiment(act_df_cut,sample_n=sample_size,permutation=True,dep_col='churned')
    if results_pdf.empty:
        results_pdf = temp_df
    else:
        results_pdf = pd.concat([results_pdf,temp_df],ignore_index=True)

In [ ]:
results_pdf.mean()

#### 2. Bootstrap tests

In [ ]:
# Bootstrap test
#sample_size = act_df_cut.groupby('variant_id')['account_number'].nunique().min()
sample_size = 200
nperms = 100
results_bdf = pd.DataFrame()
for i in range(1,100):
    temp_df = hp.prop_experiment(act_df_cut,sample_n=sample_size,dep_col='churned')
    if results_pdf.empty:
        results_bdf = temp_df
    else:
        results_bdf = pd.concat([results_bdf,temp_df],ignore_index=True)

In [ ]:
results_bdf.mean()